## QUERY LOCALIZER EXPERIMENT

Целью данного эксперимента является нахождение позы нового изображения на основе уже имеющейся реконструкции SfM. Здесь описан пайплайн, когда мы сами создаем SfM при помощи аппрата PixSfM. Вы можете также использовать уже готовую SfM.

# Libraries

In [ ]:
%load_ext autoreload
%autoreload 2
import tqdm, tqdm.notebook
tqdm.tqdm = tqdm.notebook.tqdm  # notebook-friendly progress bars

from pathlib import Path
import pycolmap
import sys
import os

sys.path.append("/workspace/pixel-perfect-sfm/")
sys.path.append("/workspace/pixel-perfect-sfm/Hierarchical-Localization")

from hloc import extract_features, match_features, reconstruction, pairs_from_exhaustive, visualization
from hloc.visualization import plot_images, read_image
from hloc.utils.viz_3d import init_figure, plot_points, plot_reconstruction, plot_camera_colmap
from hloc.utils.read_write_model import  write_next_bytes, Point3D, Image, read_images_text, read_points3D_binary,\
        write_points3D_binary, write_images_binary, read_images_binary, write_images_text, read_cameras_binary, \
        Camera, write_cameras_text, read_cameras_text

from pixsfm.util.visualize import init_image, plot_points2D
from pixsfm.refine_hloc import PixSfM
from pixsfm import ostream_redirect

from typing import Optional, List, Dict, Any

from utils import modified_write_images_text

import numpy as np
from matplotlib import pyplot as plt

import open3d as o3d
assert o3d.__version__ == '0.15.2', 'The version 0.15.2 is required!'

# redirect the C++ outputs to notebook cells
cpp_out = ostream_redirect(stderr=True, stdout=True)
cpp_out.__enter__()

# Setup

В **object_name** необходимо задать имя объекта, над которым вы хотите провести эксперимент.

**check_for_calibrated_images** - булевая переменная, по которой мы выбираем какие поз камер использовать (менее точные (True) или точные (False))

**delete_previous_output** - если True, то удаляет все предыдущие файлы в папке outputs. Использовать супер осторожно.

In [ ]:
object_name = 'dragon'

check_for_calibrated_images = False
delete_previous_output = True

**images_all** - путь к папке со всеми изображениями

**outputs** - путь к папке со всеми результатами

**cache_init** - путь к кэш-файлу, его мы получаем во время того, когда делаем KA или BA. В этот файле хранятся featuremaps после  dense feature extraction. В среднем на одну картинку размером 2368х1952 уходит 3 минуты. Этот файл вообще нельзя трогать, поэтому мы копируем его в папку outputs для своего эксперимента и продолжаем работу.

**cache_path** - тот же файл, что cache_init, с которым мы теперь будем работать во время эксперимента.

**sfm_pairs** - файл с названиями пар изображений на каждой строке

**features** - файл с features для каждой картинки, извлеченными при помощи feature_conf

**matches** - файл с matches для каждой пары картинок, извлеченными при помощи matcher_conf

**pairs-loc.txt** - файл с названиями пар изображений на каждой строке (только на этот раз идут пары для картинок из ДБ со всеми возможными картинками из папки query)

In [ ]:
root = Path('/workspace')

images_all = root / f'datasets/sk3d/dataset/{object_name}/tis_right/rgb/undistorted/ambient@best'

images_init = root / f'datasets/sk3d/dataset/{object_name}/tis_right/rgb/images.txt'
cameras_init = root / 'datasets/sk3d/dataset/calibration/tis_right/rgb/cameras.txt'

outputs = root / (f'pixel-perfect-sfm/outputs/{object_name}/query_localizer_optimum')

if delete_previous_output:
    !rm -rf $outputs
    
outputs.mkdir(parents=True, exist_ok=True)

sfm_pairs = outputs / 'pairs-sfm.txt'
features = outputs / 'features.h5'
matches = outputs / 'matches.h5'
loc_pairs = outputs / 'pairs-loc.txt'

exp_loc = outputs / "query_localizer_v1"
exp_loc.mkdir(parents=True, exist_ok=True)

# 3D mapping and refinement

Здесь описаны возможности для настройки [**extract_features**](https://github.com/cvg/Hierarchical-Localization/blob/91f40bfd765add3b59ba7376f8579d8829f7fa78/hloc/extract_features.py#L21)

Здесь описаны возможности для настройки [**match_features**](https://github.com/cvg/Hierarchical-Localization/blob/91f40bfd765add3b59ba7376f8579d8829f7fa78/hloc/match_features.py#L17)

Здесь описан пайплайн того, как можно использовать свои кастомные [**local features**, **matcher**, **image retrieval**](https://github.com/cvg/Hierarchical-Localization/tree/91f40bfd765add3b59ba7376f8579d8829f7fa78#using-your-own-local-features-or-matcher).


In [ ]:
feature_conf = extract_features.confs['superpoint_aachen']
matcher_conf = match_features.confs['superglue']

## Create db and query images

Создаем две папки: mapping и query. В папке mapping будут лежать все те картинки, которые нужны нам для построения реконструкции.  В папке query будут находиться все те картинки, для которых мы хотим новую позы.

In [ ]:
images = root / f'pixel-perfect-sfm/dataset/{object_name}_loc_opt'
images.mkdir(parents=True, exist_ok=True)

images_extra = root / f'pixel-perfect-sfm/dataset/{object_name}_loc_opt/extra'
!rm -rf $images_extra

images_extra.mkdir(parents=True, exist_ok = True)

images_references = root / f'pixel-perfect-sfm/dataset/{object_name}_loc_opt/mapping'
!rm -rf $images_references
images_references.mkdir(parents=True, exist_ok=True)

images_queries = root / f'pixel-perfect-sfm/dataset/{object_name}_loc_opt/query'
!rm -rf $images_queries
images_queries.mkdir(parents=True, exist_ok=True)

In [ ]:
import shutil

ref_num = 3
img_list = sorted([str(p) for p in images_all.iterdir()])

for fn in img_list[ref_num:100]: shutil.copy(fn, str(images_extra)) 
    
for fn in img_list[0:ref_num]: shutil.copy(fn, str(images_references))      

In [ ]:
!echo "All image references: " && ls $images_references
!echo "All image queries: " && ls $images_queries
!echo "All image extra: " && ls $images_extra

In [ ]:
_cams = read_cameras_text(str(cameras_init))
f, cx, cy, k = _cams[0].params

# https://github.com/colmap/colmap/blob/dev/src/base/camera_models.h#L201

opts = dict(camera_model='PINHOLE', camera_params=','.join(map(str, (f, cx, cy, k))))

mapper_opts = dict(ba_refine_focal_length=False, 
                       ba_refine_extra_params=False)

In [ ]:
try:
    import hloc
except ImportError:
    print("Could not import hloc.")
    hloc = None

In [ ]:
def run_triangulation(exp_loc: Path,
                      keypoints_path: Path, 
                      sfm_pairs: Path,
                      matches: Path,
                      images_references: Path,
                     ) -> pycolmap.Reconstruction:

    hloc_path = exp_loc / 'hloc'
    hloc_path.mkdir(parents=True, exist_ok=True)

    database_path = hloc_path / 'database.db' 
    reference = pycolmap.Reconstruction(exp_loc)   

    images_dict = read_images_text(exp_loc / 'images.txt')

    # Here I changed code and in database we have data about camera extrinsics    
    image_ids = hloc.triangulation.create_db_from_model(reference, 
                                                        database_path, 
                                                        images_dict)

    #Importing features into database -> keypoints table 
    hloc.triangulation.import_features(image_ids, 
                                       database_path, 
                                       keypoints_path)

    #Importing matches into database -> matches table
    skip_geometric_verification = False
    hloc.triangulation.import_matches(image_ids, 
                                      database_path, 
                                      sfm_pairs, 
                                      matches,
                                      min_match_score=None, 
                                      skip_geometric_verification=skip_geometric_verification)

    verbose, estimate_two_view_geometries = False, False

    if not skip_geometric_verification:
            if estimate_two_view_geometries:
                hloc.triangulation.estimation_and_geometric_verification(database_path, 
                                                                         sfm_pairs, 
                                                                         verbose)
            else:
                # We are doing this part to add data to two_view_geometries table
                hloc.triangulation.geometric_verification(
                    image_ids, 
                    reference, 
                    database_path, 
                    keypoints_path, 
                    sfm_pairs, 
                    matches)

    reconstruction = hloc.triangulation.run_triangulation(hloc_path, 
                                                          database_path, 
                                                          images_references, 
                                                          reference, 
                                                          verbose)

    print('Finished the triangulation with statistics:\n%s',
                reconstruction.summary())

    # Saving result to a folder
    reconstruction.write(str(hloc_path))

#     !mkdir -p $hloc_path/model_txt/

#     !colmap model_converter \
#         --input_path $hloc_path \
#         --output_path $hloc_path/model_txt/\
#         --output_type TXT
    
    return reconstruction

In [ ]:
conf_KA = {
            "dense_features": {
                    "use_cache": False,
            },
            "KA": {
                "dense_features": {'use_cache': True}, 
                "split_in_subproblems": True,
                "max_kps_per_problem": 1000,  
            },
    }

def run_featuremetric_KA(exp_loc: Path, 
                         images_references: Path, 
                         features: Path, 
                         sfm_pairs: Path, 
                         matches: Path) -> Path:

    keypoints_path = exp_loc / "refined_keypoints.h5"
    refiner = PixSfM(conf=conf_KA)

    keypoints, ka_data, feature_manager = refiner.refine_keypoints(
        output_path = keypoints_path,
        image_dir = images_references,
        features_path = features,
        pairs_path = sfm_pairs,
        matches_path = matches,
    )
    
    return keypoints_path

In [ ]:
conf_BA = {
            "dense_features": {
                    "use_cache": False,
            },
            "BA": { 
                    "apply": True,
                    "optimizer": {
                          "refine_focal_length": False,  # whether to optimize the focal length
                          "refine_principal_point": False,  # whether to optimize the principal points
                          "refine_extra_params": False,  # whether to optimize distortion parameters
                          "refine_extrinsics": False,  # whether to optimize the camera poses
                    }
                }
}

def run_featuremetric_BA(exp_loc: Path, 
                         images_references: Path,
                         mapper_options: Optional[Dict[str, Any]] = None):
    
    # running featuremetric BA
    hloc_args = dict(camera_mode=pycolmap.CameraMode.SINGLE,
                    verbose=False,
                    image_options=mapper_options)

    sfm = PixSfM_ba(conf=conf_BA)

    model, ba_data, feature_manager = sfm.refine_reconstruction(
        output_path = exp_loc / 'hloc/model',
        input_path = exp_loc / 'hloc',
        image_dir = images_references,
    )

    print(model.summary())
    
    !colmap model_converter \
        --input_path $exp_loc/hloc/model/ \
        --output_path $exp_loc/hloc/model/ \
        --output_type TXT
    
    return model, feature_manager

In [ ]:
from pixsfm.refine_hloc import PixSfM
from pixsfm.refine_colmap import PixSfM as PixSfM_ba
import pycolmap
from pixsfm.localize import QueryLocalizer, pose_from_cluster

result_dir = exp_loc / 'result'
result_dir.mkdir(parents=True, exist_ok=True)

extra_imgs_sorted = sorted(list(images_extra.iterdir()))
all_init_imgs_dict = read_images_text(images_init)

all_result = {}
cameras_dict = {}
result_queries = {}
db_imgs_sfm = {}

#Iterate through every image in extra_images folder
for i, img in enumerate(extra_imgs_sorted):

    # 1) 2 images copied in db
    
    if i == 0:
        print("Base images in db:")
        !ls $images_references
    
    # 2) Copy 1 image to query folder
    
    query_name = Path(img).stem + '.png'
    shutil.copy(str(img), str(images_queries))
    print(f"Image {query_name} is copied to {images_queries}")
    
    ####### References list updated
    reference_images = [str(p.relative_to(images_references)) for p in images_references.iterdir()]
    len_db_images = len(reference_images)
    print("Reference images: ", len(reference_images), 
          reference_images
         )  

    ####### Queries list updated
    query_images = [str(p.relative_to(images_queries)) for p in images_queries.iterdir()]
    print("Query images: ", len(query_images), 
         query_images
         )  
    
    # 3) Find image pairs for Sfm and Match features between each other
    
    args = dict(overwrite=True) if result_queries else {}  
    
    extract_features.main(feature_conf, 
                      images_references, 
                      image_list=reference_images, 
                      feature_path=features,
                      **args)
    
    pairs_from_exhaustive.main(sfm_pairs, 
                               image_list=reference_images)
    
    match_features.main(matcher_conf, 
                        sfm_pairs, 
                        features=features, 
                        matches=matches, 
                        **args);
    
    dst_file = str(exp_loc / 'images.txt')
    if os.path.exists(dst_file):
        !rm -r $dst_file
    !cp -r $images_init $exp_loc        
    print("images.txt copied!")

    dst_file = str(exp_loc / 'cameras.txt')
    if os.path.exists(dst_file):
        !rm -r $dst_file
    !cp -r $cameras_init $exp_loc
    print("cameras.txt copied!")

    !touch $exp_loc/points3D.txt
    print("points3D.txt created!")

    !ls -la $exp_loc
    
    insert = 0
    for k, img in all_init_imgs_dict.items():
        if insert == ref_num:
            break
        if img.name in reference_images:
            db_imgs_sfm[k] = img
            insert += 1

    
    # 4) Build SfM 
    
    if result_queries:
        # SfM + new results
        print("Get images and cameras to update initial SfM (SfM + queries).")
        images_dict_prev = read_images_text(str(exp_loc / 'hloc/model/images.txt'))
        db_imgs_sfm = {**images_dict_prev,**result_queries}
        
        modified_write_images_text(db_imgs_sfm, exp_loc / 'images.txt')
        
    else:     
        # initial SfM
        print("Get images and cameras for initial SfM.")
        write_images_text(db_imgs_sfm, exp_loc / 'images.txt')    
        cameras_dict = read_cameras_text(exp_loc / 'cameras.txt')   
        
    keypoints_path = run_featuremetric_KA(exp_loc=exp_loc, 
                                          images_references=images_references, 
                                          features=features, 
                                          sfm_pairs=sfm_pairs, 
                                          matches=matches)
    

    model_KA = run_triangulation(exp_loc=exp_loc, 
                     keypoints_path=keypoints_path, 
                     sfm_pairs=sfm_pairs, 
                     matches=matches, 
                     images_references=images_references)

    model_BA, feature_manager = run_featuremetric_BA(exp_loc=exp_loc, 
                                                  images_references=images_references,
                                                  mapper_options=mapper_opts)
    
    # 5) Extract features for query image
    
    features_query = outputs / 'features_query.h5'
    !rm -rf $features_query 
    
    matches_query = outputs / 'matches_query.h5'
    !rm -rf $matches_query

    extract_features.main(feature_conf, 
                          images_queries, 
                          image_list=query_images, 
                          feature_path=features_query)
    
    references_registered = [model_BA.images[i].name for i in model_BA.reg_image_ids()]

    pairs_from_exhaustive.main(loc_pairs, 
                           image_list=query_images, 
                           ref_list=references_registered)
    
    match_features.main(matcher_conf, 
                        loc_pairs, 
                        features=features_query, 
                        matches=matches_query, 
                        features_ref=features);
       
    
    # 6) Run pose localization
    
    ref_ids = [model_BA.find_image_with_name(r).image_id for r in references_registered]
    
    loc_conf = {
        #"dense_features": model_BA.conf.dense_features, 
        # same features as the SfM refinement
        "PnP": {  # initial pose estimation with PnP+RANSAC
            'estimation': {'ransac': {'max_error': 12.0}},
            'refinement': {'refine_focal_length': False, 
                           'refine_extra_params': False},
        },
        "QBA": {  # query pose refinement
            "optimizer:": {'refine_focal_length': False, 
                           'refine_extra_params': False},
        }
    }

    localizer = QueryLocalizer(model_BA, 
                               conf=loc_conf, 
                               dense_features=feature_manager)
    
    
    _cams = read_cameras_binary(str(exp_loc / 'hloc/model/cameras.bin'))
    cam_info = _cams[0]

    pinhole_camera = pycolmap.Camera(
                                model='PINHOLE',
                                width=cam_info.width,
                                height=cam_info.height,
                                params=cam_info.params)

    print("Camera info --> ", pinhole_camera)


    # 7) Save results from experiment (we got new pose for query image here) and delete file with results 
    
    ret, log = pose_from_cluster(localizer, 
                             query_name, 
                             pinhole_camera, 
                             ref_ids, 
                             features_query, 
                             matches_query, 
                             image_path=images_queries / query_name)
    
    print(f"ret --> {query_name}", ret['qvec'], ret['tvec'], ret['camera'])
    
    
    
    image_id = int(Path(query_name).stem) + 1
    
    result_queries[image_id] = Image(
                id=image_id, 
                qvec=ret['qvec'], 
                tvec=ret['tvec'],
                camera_id=cam_info.id, 
                name=query_name,
                xys= np.array([]), 
                point3D_ids= np.array([]),
    )
    
    # 9) Delete query from query folder
    query_to_delete = images_queries / query_name
    query_to_delete.unlink()  
    
    # 10) Copy this query image to db folder for next iteration run
    shutil.copy(str(images_all / query_name), str(images_references))


In [ ]:
result_queries